## Trabajo PLN I. Corrector ortográfico

### Autores:

   **- Porras Naranjo, Miguel Ángel**
   
   **- Terrón Dastis, Pedro**

Volveremos a usar la función `word_tokenize` de la librería `nltk`.

In [5]:
from nltk import word_tokenize

Basandonos en el siguiente corpus (https://github.com/javierarce/palabras/blob/master/listado-general.txt), cargamos todas las palabras del diccionario español. El sistema está creado para cualquier idioma, siempre que se tenga un listado de todas sus palabras.

In [6]:
f = open('listado-general-palabras-esp.txt', encoding="utf-8")
dic_raw = f.read()
f.close()

In [7]:
dic_esp = word_tokenize(dic_raw)

El modelo que vamos a crear se basa en funciones de distancia. Concretamente definiremos 3 funciones que relacionamos con los errores más comunes a la hora de escribir.

+ Distancia 1 (`d1`): Errores del tipo omitir una letra. Por ejemplo, "huevo" con "uevo" y "tenedor" con "tenedo".
+ Distancia 2 (`d2`): Errores del tipo añadir una letra de más. Por ejemplo, "cuarto" con "chuarto".
+ Distancia 3 (`d3`): Errores del tipo confundir una letra con otra. Por ejemplo, "gitano" con "jitano".

Para este algoritmo, `d1` y `d2` reciben dos palabras, la que esta mal escrita, y una que podria ser su corrección. La función devolverá `True` o `False`.

La función `d3` devolverá una lista de todas las palabras que se parezcan a la incorrecta.



In [8]:
def d1(word,correct_word):
    if len(correct_word) == 1+len(word):
        for i in range(len(correct_word)):
            if (word == correct_word[:i] + correct_word[i+1 :]):
                return(True)
        return(False)
    else:
        return(False)

In [9]:
d1("caa","casa")

True

In [10]:
d1("cushara", "tenedor")

False

In [11]:
def d2(word,correct_word):
    return(d1(correct_word,word))

In [12]:
d2("huesol","hueso")

True

In [13]:
d2("esternocleidomastoideo","supercalifragilisticoespialidoso")

False

In [14]:
dic_letras = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","ñ","o","p","q","r","s","t","u",
              "v","w","x","y","z"]


def d3(word,dic=dic_letras):
    l =  []
    for letter in dic:
        for i in range(len(word)):
            word_test = (word[:i] + letter + word[i+1 :])
            if word_test in dic_esp:
                l.append(word_test)
    return(l)


In [15]:
d3("cawa")

['caca',
 'cada',
 'caja',
 'cala',
 'cama',
 'cana',
 'caña',
 'capa',
 'cara',
 'casa',
 'cata',
 'cava',
 'caya',
 'caza']

Implementamos la función que devuelve toda la lista de posibles correcciones de la palabra, si esta no se encuentra en el diccionario

In [16]:
def corrector_word(word,dic=dic_esp):
    l = []
    if word in dic_esp:
        return([])
    else:
        for correct_word in dic:
            if d1(correct_word,word):
                l.append(correct_word)
            if d2(word,correct_word):
                l.append(correct_word)
        arreglo = d3(word)
        l += arreglo
        return(set(l))

In [18]:
corrector_word("vonito")

{'bonito'}

In [19]:
corrector_word("edifficio")

{'edificio'}

In [24]:
corrector_word("esclalera")

{'escalera'}

Y definimos la función `corrector_phase` que analiza todas las palabras por separado mediante `corrector_word` y en caso de que sean incorrectas, devuelve una lista de posibles correcciones. 

In [25]:
def corrector_phrase(phrase):
    word_list = word_tokenize(phrase)
    i = 0
    for word in word_list:
        correct_words = corrector_word(word)
        if correct_words != []:
            i = 1
            print("La palabra '{0}' es incorrecta, una posible corrección seria: '{1}' .".format(word,correct_words))
    if i == 0:
        print("La frase está bien escrita")
    
            

In [27]:
corrector_phrase("eld pizo ez mui jrande")

La palabra 'eld' es incorrecta, una posible corrección seria: '{'el', 'ele'}' .
La palabra 'pizo' es incorrecta, una posible corrección seria: '{'pino', 'piso', 'pazo', 'pijo', 'piño', 'pico', 'pilo', 'pipo', 'mizo', 'pito', 'lizo', 'rizo', 'pozo', 'tizo', 'piro'}' .
La palabra 'ez' es incorrecta, una posible corrección seria: '{'z', 'ex', 'et', 'e', 'eh', 'el', 'en', 'ea'}' .
La palabra 'mui' es incorrecta, una posible corrección seria: '{'tui', 'mus', 'mu', 'cui', 'qui', 'mi', 'muy', 'mur'}' .
La palabra 'jrande' es incorrecta, una posible corrección seria: '{'grande'}' .


En caso de que hubiera alguna palabra que no estuviera en el diccionario, podemos usar la función `add_word` para añadirla, y así recibirla en futuras sugerencias.

In [40]:
def add_word(word, dic=dic_esp):
    dic = dic_esp.append(word)

In [41]:
add_word("bonita")